In [ ]:
from transformers import pipeline
import pandas as pd
import torch

# **Model Preparation:**

In [ ]:
!git clone https://github.com/AI4Bharat/indic-asr-api-backend.git

Cloning into 'indic-asr-api-backend'...
remote: Enumerating objects: 111, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (69/69), done.
remote: Total 111 (delta 26), reused 92 (delta 18), pack-reused 0
Receiving objects: 100% (111/111), 27.80 KiB | 4.63 MiB/s, done.
Resolving deltas: 100% (26/26), done.
Filtering content: 100% (15/15), 2.43 GiB | 20.93 MiB/s, done.


In [ ]:
!mkdir -p models
!wget -P models https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/all_lang_models/hindi_models.zip
!unzip -q /content/models/hindi_models.zip -d models/

--2024-04-26 09:13:19--  https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/all_lang_models/hindi_models.zip
Resolving objectstore.e2enetworks.net (objectstore.e2enetworks.net)... 164.52.210.57, 164.52.210.96, 164.52.210.58, ...
Connecting to objectstore.e2enetworks.net (objectstore.e2enetworks.net)|164.52.210.57|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4262614425 (4.0G) [application/zip]
Saving to: ‘models/hindi_models.zip’

hindi_models.zip    100%[===================>]   3.97G  40.9MB/s    in 1m 41s  

2024-04-26 09:15:02 (40.1 MB/s) - ‘models/hindi_models.zip’ saved [4262614425/4262614425]



In [ ]:
model = pipeline(
    task="automatic-speech-recognition",
    model="/content/models/hindi_models/whisper-large-hi-noldcil",
    device=torch.device("cuda:0"),
    chunk_length_s=30,
    generate_kwargs={"num_beams": 5}
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# **Prepare Dataset:**

In [ ]:
!wget https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar/kathbath.zip
!unzip -q /content/kathbath.zip

--2024-04-26 09:01:35--  https://objectstore.e2enetworks.net/indic-asr-public/indicwhisper/vistaar/kathbath.zip
Resolving objectstore.e2enetworks.net (objectstore.e2enetworks.net)... 164.52.206.154, 164.52.210.96, 101.53.136.18, ...
Connecting to objectstore.e2enetworks.net (objectstore.e2enetworks.net)|164.52.206.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9187772731 (8.6G) [application/zip]
Saving to: ‘kathbath.zip’

kathbath.zip        100%[===================>]   8.56G  35.4MB/s    in 3m 48s  

2024-04-26 09:05:25 (38.5 MB/s) - ‘kathbath.zip’ saved [9187772731/9187772731]



In [ ]:
result = model("/content/kathbath/hindi/test_known/audio/1052/844424933454281-1052-m.wav")

In [ ]:
print(result["text"])

हेमचंद्र के अनुकार्थ संग्रह में यशस्वी विस्तारित परशादी कर्द में पीच दिया गया है


In [ ]:
def pred_rec(path):
  result = model(path)
  pred = result["text"]
  return pred

In [ ]:
kabath_df = pd.read_csv('/content/kathbath/hindi/test_known/bucket.csv')
kabath_df['file_path'] = kabath_df['file_path'].astype("string")
kabath_df['file_path'] = kabath_df['file_path'].str.replace('/nlsasfs/home/ai4bharat/gramesh/speechteam/e2e_data/pilot4_karya/aug_dc_data_raw/dc_data_clean_wav', '/content/kathbath')
kabath_rand = kabath_df.sample(n = 10)

In [ ]:
kabath_rand

,file_path,transcript,length
1480,/content/kathbath/hindi/test_known/audio/888/8...,कंपनी अभी इस फीचर के बीटा वर्जन को टेस्ट कर रह...,53499
1809,/content/kathbath/hindi/test_known/audio/1160/...,दोनों देशों के बीच ये मुकाबला ल्यूज्ह्निकी स्ट...,62415
2388,/content/kathbath/hindi/test_known/audio/266/8...,चर्चिल वार मंत्रालय के साथ पार्टी एक बार फिर १...,100310
1197,/content/kathbath/hindi/test_known/audio/727/8...,इस बार मराठी मुद्दे को लेकर यह रिएलिटी शो विवा...,67617
1329,/content/kathbath/hindi/test_known/audio/888/8...,हालांकि सभा स्थल का चयन अभी नहीं किया जा सका है,57214
984,/content/kathbath/hindi/test_known/audio/1102/...,फिल्मकार आदित्य चोपड़ा ने भी शनिवार को वरसोवा ...,100682
329,/content/kathbath/hindi/test_known/audio/384/8...,इसके खिलाफ देश की जनता में भारी रोष है,52756
2716,/content/kathbath/hindi/test_known/audio/705/8...,बिजली के संचरण और वितरण में ऊर्जा उत्पाद महत्व...,108112
2516,/content/kathbath/hindi/test_known/audio/390/8...,मुस्लिम समाज के बच्चो के लिए मदरसा भी है,93995
436,/content/kathbath/hindi/test_known/audio/326/8...,गांधी जी ने देशवासियों को स्वच्छता का संदेश दि...,68731


In [ ]:
kabath_rand['predict'] = kabath_rand.iloc[:10, 0].apply(pred_rec)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
kabath_rand

,file_path,transcript,length,predict
1480,/content/kathbath/hindi/test_known/audio/888/8...,कंपनी अभी इस फीचर के बीटा वर्जन को टेस्ट कर रह...,53499,कंपनियां भी इस फीचर के बीटा वर्जन को टेस्ट कर ...
1809,/content/kathbath/hindi/test_known/audio/1160/...,दोनों देशों के बीच ये मुकाबला ल्यूज्ह्निकी स्ट...,62415,दोनों देशों के बीच यह मुकाबला लियूज़ इन के स्ट...
2388,/content/kathbath/hindi/test_known/audio/266/8...,चर्चिल वार मंत्रालय के साथ पार्टी एक बार फिर १...,100310,चर्चिल वार मंत्रालय के साथ पार्टी एक बार फिर ए...
1197,/content/kathbath/hindi/test_known/audio/727/8...,इस बार मराठी मुद्दे को लेकर यह रिएलिटी शो विवा...,67617,इस बार मराठी मुद्दे को लेकर यह रियल्टी शो विवा...
1329,/content/kathbath/hindi/test_known/audio/888/8...,हालांकि सभा स्थल का चयन अभी नहीं किया जा सका है,57214,हालांकि सभा स्थल का चयन अभी नहीं किया जा सका है
984,/content/kathbath/hindi/test_known/audio/1102/...,फिल्मकार आदित्य चोपड़ा ने भी शनिवार को वरसोवा ...,100682,फिल्मकार आदित्य चोपड़ा ने भी शनिवार को वर्सोवा...
329,/content/kathbath/hindi/test_known/audio/384/8...,इसके खिलाफ देश की जनता में भारी रोष है,52756,इनके खिलाफ देश की जनता में भारी रोष है
2716,/content/kathbath/hindi/test_known/audio/705/8...,बिजली के संचरण और वितरण में ऊर्जा उत्पाद महत्व...,108112,बिजली के संचरण और वितरण में ऊर्जा उत्पाद महत्व...
2516,/content/kathbath/hindi/test_known/audio/390/8...,मुस्लिम समाज के बच्चो के लिए मदरसा भी है,93995,मुस्लिम समाज के बच्चों के लिए मदरसा भी है
436,/content/kathbath/hindi/test_known/audio/326/8...,गांधी जी ने देशवासियों को स्वच्छता का संदेश दि...,68731,गांधीजी ने देशवासियों को स्वच्छता का संदेश दिय...


In [ ]:
kabath_rand.iloc[:10, 1].to_csv('kabath_ref.txt', sep='\t', index=False, header = None)
kabath_rand.iloc[:10, 3].to_csv('kabath_hyp.txt', sep='\t', index=False, header = None)

# **ASR Evaluation:**

In [ ]:
!pip install asr-evaluation

In [ ]:
!wer /content/kabath_ref.txt /content/kabath_hyp.txt

Sentence count: 10
WER:    15.517% (        18 /        116)
WRR:    89.655% (       104 /        116)
SER:    80.000% (         8 /         10)
